In [ ]:
import rtsvg
rt = rtsvg.RACETrack()
import ollama
from pydantic import BaseModel
import time
import os

class NumberClueOrientation(BaseModel):
    number:      int
    clue:        str
    orientation: str

class CrosswordPuzzleEntries(BaseModel):
    items: list[NumberClueOrientation]

model    = 'qwen2.5vl:7b' # 'qwen2.5vl:32b' # 'llama4:scout' # 'qwen2.5vl:7b' # 'gemma3:27b' # 'qwen2.5vl:32b' # 'gemma3:12b' 'llama3.2-vision:11b' #
client   = ollama.Client()

_lu_           = {}
_process_time_ = {}

_screenshot_dir_ = '../../../data/crossword_puzzle_screenshots/'
_files_ = os.listdir(_screenshot_dir_)

In [ ]:
if False:
    _prompt_ = """Extract the crossword entries."""
    for _file_ in _files_:
        screenshots = []
        if _file_.endswith('.jpeg'): 
            screenshots.append(os.path.join(_screenshot_dir_, _file_))
            print(screenshots)
            t0 = time.time()
            response = client.chat(model=model, messages=[
                {
                    'role':    'user',
                    'content': _prompt_,
                    'images':  screenshots,
                },],
                format=CrosswordPuzzleEntries.model_json_schema(),
                options={'num_ctx': 8192+2048}
            )
            t1 = time.time()
            _items_ = CrosswordPuzzleEntries.model_validate_json(response['message']['content'])
            _lu_[_file_], _process_time_[_file_] = _items_, t1-t0

In [ ]:
#
# Group By number and orientation -- and into a set to see where there are duplicates (or discrepancies)
#
entry_to_clues = {}
entry_to_files = {}
for x in _lu_:
    for y in _lu_[x].items:
        entry = (y.number, y.orientation.lower())
        if entry not in entry_to_clues: 
            entry_to_clues[entry] = set()
            entry_to_files[entry] = set()
        entry_to_clues[entry].add(y.clue)
        entry_to_files[entry].add(x)

In [ ]:
#
# Just dump this to a file and fix by hand
# ... first part is dump / second part is load
#

#_tuples_ = list(entry_to_clues.keys())
#print('{')
#for x in sorted(_tuples_, key=lambda x: (x[1], x[0])): print(x, ':', entry_to_clues[x], ',')
#print('}')

for _file_ in _files_:
    if _file_.endswith('_entries.txt') and _file_.startswith('.') == False:
        print(_file_)
        with open(os.path.join(_screenshot_dir_, _file_), 'rt') as f:
            crossword_entries = eval(f.read())

In [ ]:
#
# The following produced something that may be usable... ... wasn't usable ... was just a grid...
# Model:  'qwen2.5vl:32b'
#
_prompt_ = """Extract out the numbered cells.  Return as a JSON object that includes the number, the x index, and the y index.
"""
'''
for _file_ in _files_:
    screenshots = []
    if _file_.endswith('.jpeg'): 
        screenshots.append(os.path.join(_screenshot_dir_, _file_))
        print(screenshots)
        t0 = time.time()
        response = client.chat(model=model, messages=[
            {
                'role':    'user',
                'content': _prompt_,
                'images':  screenshots,
            },],
            options={'num_ctx': 8192+2048}
        )
        t1 = time.time()
        print(response['message']['content'])
        break
'''

In [ ]:
#
# This doesn't work either ... the numbers are often incorrect...
# ... llama4:scout produces more variety in results ... but they are incorrect as well...
#
class NumberLocation(BaseModel):
    number: int
    x:      int
    y:      int
all_numbers = set()
for _tuple_ in crossword_entries: all_numbers.add(_tuple_[0])
screenshots = []
for _file_ in _files_:
    if _file_.endswith('.jpeg'): 
        screenshots.append(os.path.join(_screenshot_dir_, _file_))
        break
print(screenshots)
all_numxys = []
for _number_ in all_numbers:
    _prompt_ = f"""Which x and y pixel is the number {_number_} at in the crossword puzzle?  Return as a JSON object that includes the number, the x pixel, and the y pixel."""
    t0    = time.time()
    model = 'llama4:scout'
    response = client.chat(model=model, messages=[
        {
            'role':    'user',
            'content': _prompt_,
            'images':  screenshots,
        },],
        format=NumberLocation.model_json_schema(),
        options={'num_ctx': 8192+2048}
    )
    t1 = time.time()
    _num_xy_ = NumberLocation.model_validate_json(response['message']['content'])
    all_numxys.append(_num_xy_)
    print('.',end='')
    

In [ ]:
svg = [f'<svg x="0" y="0" width="800" height="800"><rect x="0" y="0" width="800" height="800" fill="#ffffff" />']
for _nl_ in all_numxys:
    n, x, y = _nl_.number, _nl_.x, _nl_.y
    svg.append(rt.svgText(str(n), x + 2, y + 2 + 15, txt_h=15, color='gray'))
svg.append('</svg>')
rt.tile([''.join(svg)])